<a href="https://colab.research.google.com/github/zwfswt/GEE/blob/main/S2_NDVI_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=hGc2Jr_tqJyUyC0Bwqdbpqx27J7F4_wKHI7FJaroNu0&tc=zZurTByRnMyc7xA3jnoKbUOGN43MS0ffFfmrhKvBQw0&cc=KWd-zEjqkkE7l-lCmqZWrG_j80ug46xyFb1Uz_VnV3Y

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgNWZJ5e5WA3nSThTGyA92g6vCkcKFUoMMOd-j1zwz4bq40Rh6Llyw

Successfully saved authorization token.


In [2]:
try:
    import geemap
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print('geemap not found, installing via pip in Google Colab...')
        !pip install geemap --quiet
        import geemap
    else:
        print('geemap not found, please install via conda in your environment')

geemap not found, installing via pip in Google Colab...
     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 29.3 MB/s 
     |████████████████████████████████| 187 kB 34.8 MB/s 
     |████████████████████████████████| 1.3 MB 35.3 MB/s 
     |████████████████████████████████| 3.0 MB 48.6 MB/s 
     |████████████████████████████████| 130 kB 73.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 95 kB 4.0 MB/s 
     |████████████████████████████████| 99 kB 9.3 MB/s 
     |████████████████████████████████| 98 kB 7.9 MB/s 
     |████████████████████████████████| 8.7 MB 55.7 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 128 kB 60.3 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 342 kB 55.2 MB/s 
     |████████████████████████████████| 428 kB 46.8 MB/s 
     |█████████████████████

In [55]:
javascript_code = """

var geometry = ee.Geometry.Polygon(
               [[124.05,32.27],
               [124.05,34.38],
               [121.54,34.38],
               [121.54,32.27]]);
var aoi = ee.FeatureCollection(geometry);
function maskS2clouds(image) {
  var qa = image.select('QA60');
  var cloudBitMask = 1 << 5;
  var cirrusBitMask = 1 << 11;
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).divide(10000);
}

var dataset = ee.ImageCollection('COPERNICUS/S2')
                  .filterDate('2022-04-27', '2022-05-06')
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                  .map(maskS2clouds);

var vis = {min: 0.1, max: 1, palette: [
  'FF0000', '00FF00'
]};

function NDVI(image){
  return image.select().addBands(image.normalizedDifference(['B8', 'B4']));
}

var median=dataset.median();
var clip=median.clipToCollection(aoi);
var NDVIImg=NDVI(clip);
Map.setCenter(121.43, 34.79,7);
Map.addLayer(NDVIImg, vis, 'NDVI');

Export.image.toDrive({
image: NDVIImg,
description: 'Sentinel-NDVI',
fileNamePrefix: 'S2_NDVI',
maxPixels: 1e13,
region: aoi
});

var rgbVis = {
  min: 0.0,
  max: 3000,
  bands: ['B4', 'B3', 'B2'], 
};

var exportImage = clipped.select('B.*')

Export.image.toDrive({
    image: exportImage,
    description: 'Bangalore_Composite_Raw',
    folder: 'earthengine',
    fileNamePrefix: 'bangalore_composite_raw',
    region: geometry,
    scale: 20,
    maxPixels: 1e9
})

"""

In [56]:
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

import ee
import geemap
Map = geemap.Map()

geometry = ee.Geometry.Polygon(
               [[124.05,32.27],
               [124.05,34.38],
               [121.54,34.38],
               [121.54,32.27]])
aoi = ee.FeatureCollection(geometry)
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 5
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

dataset = ee.ImageCollection('COPERNICUS/S2') \
                  .filterDate('2022-04-27', '2022-05-06') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                  .map(maskS2clouds)

vis = {'min': 0.1, 'max': 1, 'palette': [
  'FF0000', '00FF00'
]}

def NDVI(image):
  return image.select().addBands(image.normalizedDifference(['B8', 'B4']))

median=dataset.median()
clip=median.clipToCollection(aoi)
NDVIImg=NDVI(clip)
Map.setCenter(121.43, 34.79,7)
Map.addLayer(NDVIImg, vis, 'NDVI

显示RGB图像

In [ ]:
import ee
import geemap
Map = geemap.Map()

geometry = ee.Geometry.Polygon(
               [[124.05,32.27],
               [124.05,34.38],
               [121.54,34.38],
               [121.54,32.27]])
# aoi = ee.FeatureCollection(geometry)
Map.centerObject(geometry,7)
s2 = ee.ImageCollection("COPERNICUS/S2")
# rgbVis = {
#   'min': 0.0,
#   'max': 3000,
#   'bands': ['B8', 'B4', 'B3']
# }
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2022-05-03', '2022-05-05')) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

composite = withNdvi.median()
ndviComposite = composite.select('ndvi').clip(geometry)
palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']
ndviVis = {'min':0, 'max':0.5, 'palette': palette }
# Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')
# Map.addLayer(ndviComposite, ndviVis, 'NDVI Composite')
# Map
rgbVis1 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}
rgbVis2 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B8', 'B4', 'B3']
}
# image = filtered.median(); 
image = filtered.mosaic() 
clipped = image.clip(geometry)
# Map.addLayer(clipped, rgbVis1, 'RGB')
Map.addLayer(clipped, rgbVis2, 'FalseColor')
Map 

导出RGB图像

In [76]:
task = ee.batch.Export.image.toDrive(**{
    'image': clipped.select('B.*'),
    'description': 'Image Export {}'.format(i+1),
    'fileNamePrefix': image.id().getInfo(),
    'folder':'earthengine',
    'scale': 100,
    'region': image.geometry().bounds().getInfo()['coordinates'],
    'maxPixels': 1e10
  })
task.start()
print('Started Task: ', i+1)

Started Task:  7


监测导出任务

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

取消任务

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
    task_id = task.status()['id']
    task_state = task.status()['state']
    if task_state == 'RUNNING' or task_state == 'READY':
        task.cancel()
        print('Task {} canceled'.format(task_id))
    else:
        print('Task {} state is {}'.format(task_id, task_state))

显示NDVI

In [ ]:
import ee
import geemap
Map = geemap.Map()

geometry = ee.Geometry.Polygon(
               [[124.05,32.27],
               [124.05,34.38],
               [121.54,34.38],
               [121.54,32.27]])
# aoi = ee.FeatureCollection(geometry)
Map.centerObject(geometry,7)
s2 = ee.ImageCollection("COPERNICUS/S2")
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B8', 'B4', 'B3']
}
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2 \
  .filter(ee.Filter.date('2022-05-03', '2022-05-05')) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

composite = withNdvi.median()
ndviComposite = composite.select('ndvi').clip(geometry)
palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']
ndviVis = {'min':0, 'max':0.5, 'palette': palette }
# Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')
Map.addLayer(ndviComposite, ndviVis, 'NDVI Composite')
Map

In [37]:
image_ids = withNdvi.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  0


In [30]:
# Export with 100m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index', image_id)).first())
  task = ee.batch.Export.image.toDrive(**{
    'image': image.select('ndvi'),
    'description': 'Image Export {}'.format(i+1),
    'fileNamePrefix': image.id().getInfo(),
    'folder':'earthengine',
    'scale': 10,
    'region': image.geometry().bounds().getInfo()['coordinates'],
    'maxPixels': 1e10
  })
  task.start()
  print('Started Task: ', i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7


管理正在运行/等待的任务

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

取消任务

In [ ]:
# tasks = ee.batch.Task.list()
# for task in tasks:
#     task_id = task.status()['id']
#     task_state = task.status()['state']
#     if task_state == 'RUNNING' or task_state == 'READY':
#         task.cancel()
#         print('Task {} canceled'.format(task_id))
#     else:
#         print('Task {} state is {}'.format(task_id, task_state))